In [13]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
%pip install sklearn
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\012883631\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


     -------------------------------------- 292.8/292.8 KB 2.6 MB/s eta 0:00:00


You should consider upgrading via the 'c:\Users\012883631\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



     ---------------------------------------- 7.2/7.2 MB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 55.3/55.3 KB 1.5 MB/s eta 0:00:00
     -------------------------------------- 930.9/930.9 KB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 40.8/40.8 KB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 98.3/98.3 KB 5.5 MB/s eta 0:00:00


In [14]:
df = pd.read_csv('./data/diabetes_012_health_indicators_BRFSS2015.csv')

In [23]:
x_data = df.drop(["Diabetes_012"],axis=1)
y_data = df["Diabetes_012"]
MinMaxScaler = preprocessing.MinMaxScaler()
X_data_minmax = MinMaxScaler.fit_transform(x_data)
data = pd.DataFrame(X_data_minmax,columns=x_data.columns)

X_train, X_test, y_train, y_test = train_test_split(data, y_data,test_size=0.2, random_state = 1)
knn_clf=KNeighborsClassifier()
knn_clf.fit(X_train,y_train)
ypred=knn_clf.predict(X_test)

In [24]:
from sklearn.metrics import classification_report, accuracy_score

result1 = classification_report(y_test, ypred)
print("Classification Report: ")
print (result1)
result2 = accuracy_score(y_test,ypred)
print("Accuracy:",result2)

Classification Report: 
              precision    recall  f1-score   support

         0.0       0.86      0.95      0.90     42798
         1.0       0.00      0.00      0.00       873
         2.0       0.38      0.18      0.25      7065

    accuracy                           0.83     50736
   macro avg       0.41      0.38      0.38     50736
weighted avg       0.78      0.83      0.80     50736

Accuracy: 0.8283467360454115


In [27]:
#y_data[y_data == 1]

64        1.0
81        1.0
193       1.0
236       1.0
275       1.0
         ... 
253528    1.0
253529    1.0
253566    1.0
253664    1.0
253666    1.0
Name: Diabetes_012, Length: 4631, dtype: float64